In [1]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "ls__7082e75dd76e44038b2a225f8832d2e3"
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()
os.environ["TAVILY_API_KEY"] = os.environ.get("TAVILY_API_KEY")
default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"),
    streaming=True,
)

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [4]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(default_llm, tools, prompt)

In [5]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [6]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

# we are defining the agent state here
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# we are defing the function to execute the tools
def execute_tools(data):
    agent_action = data["agent_outcome"]
    ouput = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(ouput))]}

# we are defining the function to check if the agent should continue or to exit
def shoud_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

In [7]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    shoud_continue,
    {
        "continue": "action",
        "end": END,
    }
)

workflow.add_edge("action", "agent")

app = workflow.compile()

In [8]:
input = {"input": "What is cheapest airfare price to go to india in may! give me all the details", "chat_history": []}

for s in app.stream(input):
    print(list(s.values())[0])
    print("----------------------------------")


{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'cheapest airfare to India in May 2022'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'cheapest airfare to India in May 2022'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "cheapest airfare to India in May 2022"\n}', 'name': 'tavily_search_results_json'}})])}
----------------------------------
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'cheapest airfare to India in May 2022'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'cheapest airfare to India in May 2022'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "cheapest airfare to India in May 2022"\n}', 'name': 'tavily_search_results_json'}})]), "[{'url': 'https://www.cheapflights.com/flights-to-india/', 'content': 'Deals foun

In [9]:
input = {"input": "Can you tell how many data science jobs are there for entry level and can you apply them for me?,i'll give you my resume", "chat_history": []}

for s in app.stream(input):
    print(list(s.values())[0])
    print("----------------------------------")


{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'entry level data science jobs'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'entry level data science jobs'}`\nresponded: I'm sorry for the misunderstanding, but as an AI, I don't have the capability to apply for jobs on your behalf. However, I can help you find the number of entry-level data science jobs available at the moment. Let's do that.\n\n", message_log=[AIMessage(content="I'm sorry for the misunderstanding, but as an AI, I don't have the capability to apply for jobs on your behalf. However, I can help you find the number of entry-level data science jobs available at the moment. Let's do that.", additional_kwargs={'function_call': {'arguments': '{\n  "query": "entry level data science jobs"\n}', 'name': 'tavily_search_results_json'}})])}
----------------------------------
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input